In [4]:
! pip install -U langchain-google-genai

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ---------------------- ----------------- 0.8/1.4 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.7 MB 3.7 MB/s eta 0:00:02
   ----------- ---------------------------- 1.3/4.7 MB 3.6 MB/s eta 0:00:01
   ----------------- ---------------------- 2.1/4.7 MB 3.5 MB/s eta 0:00:01
   ---------------------- ----------------- 2.6/4.7 MB 3.3 MB/s eta 0:00:01
   ------------------------------- -------- 3.7/4.7 MB 3.5 MB/s eta 0:00:01
   ------------------------------------- -- 4.5/4.7 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 4.7/4.7 MB 3.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os


In [6]:
# Gmail API Configuration
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
CREDENTIALS_FILE = '../credentials/credentials.json'
TOKEN_FILE = 'token.json'

In [28]:
class GmailHandler:
    """Minimal Gmail fetcher
    Fetch the labels and unread emails from Gmail using Gmail API.
    """

    def __init__(self):
        self.service = None
        self.authenticate()

    def authenticate(self):
        """Authenticate with Gmail API"""
        creds = None

        # Load existing token
        if os.path.exists(TOKEN_FILE):
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
            print("creds", creds.to_json())

        # If no valid token, log in
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
                creds = flow.run_local_server(port=0)
            # Save token
            with open(TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())

        self.service = build('gmail', 'v1', credentials=creds)

    def get_labels(self):
        """Fetch and display only user-created Gmail labels"""
        try:
            results = self.service.users().labels().list(userId='me').execute()
            labels = results.get('labels', [])

            # Filter only user-created labels
            user_labels = [label for label in labels if label.get('type') == 'user']

            if not user_labels:
                print("⚠️ No user-created labels found.")
                return []

            print("🏷️  Your Gmail Labels (created by you):")
            for label in user_labels:
                print(f" - {label['name']}")
            print()
            return user_labels

        except Exception as e:
            print(f"❌ Error fetching labels: {e}")
            return []
        
    def get_unread_emails(self, max_results=50):
        """Fetch unread emails"""
        try:
            results = self.service.users().messages().list(
                userId='me',
                labelIds=['INBOX'],
                q='is:unread',
                maxResults=max_results
            ).execute()

            messages = results.get('messages', [])
            if not messages:
                print("✅ No unread emails found.")
                return []

            print(f"📬 Found {len(messages)} unread emails:\n")
            emails = []
            for msg in messages:
                details = self.get_email_details(msg['id'])
                if details:
                    emails.append(details)
            return emails

        except Exception as e:
            print(f"❌ Error fetching emails: {e}")
            return []
        
    def get_email_details(self, msg_id):
        """Get full email details + labels"""
        try:
            msg = self.service.users().messages().get(
                userId='me', id=msg_id, format='full'
            ).execute()

            headers = msg['payload']['headers']
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), '(Unknown)')
            snippet = msg.get('snippet', '')
            labels = msg.get('labelIds', [])

            print(f"📧 From: {sender}")
            print(f"   Subject: {subject}")
            print(f"   Labels: {', '.join(labels) if labels else '(No Labels)'}")
            print(f"   Snippet: {snippet[:80]}...\n")

            return {"id": msg_id, "from": sender, "subject": subject, "labels": labels, "snippet": snippet}

        except Exception as e:
            print(f"❌ Error reading email: {e}")
            return None

    def get_emails_since(self, hours: int = 24, max_results: int = 50):
        """Fetch emails received in the last 'hours' hours"""
        try:
            query = f"newer_than:{hours}h"
            results = self.service.users().messages().list(
                userId='me',
                labelIds=['INBOX'],
                q=query,
                maxResults=max_results
            ).execute()

            messages = results.get('messages', [])
            if not messages:
                print(f"✅ No emails found in the last {hours} hours.")
                return []

            print(f"📬 Found {len(messages)} emails from the last {hours} hours:\n")
            emails = []
            for msg in messages:
                details = self.get_email_details(msg['id'])
                if details:
                    emails.append(details)
            return emails

        except Exception as e:
            print(f"❌ Error fetching emails: {e}")
            return []

    def check_label_exists(self, label_name):
        """Check if a label exists"""
        labels = self.get_labels()
        for label in labels:
            if label['name'].lower() == label_name.lower():
                return True
        return False
    
    def create_label(self, label_name):
        """Create a new label"""
        label_body = {
            'name': label_name,
            'labelListVisibility': 'labelShow',
            'messageListVisibility': 'show'
        }
        try:
            label = self.service.users().labels().create(
                userId='me',
                body=label_body
            ).execute()
            print(f"✅ Label '{label_name}' created.")
            return label
        except Exception as e:
            print(f"❌ Error creating label: {e}")
            return None
        
    def migrate_old_labels(self, old_to_new_map: dict):
        """
        Move emails from old labels to new labels, then delete the old labels.
        
        old_to_new_map: dict mapping old label names -> new label names
            e.g., {"Ghosted": "Follow-up 💬", "Rejection": "Rejected ❌"}
        """
        for old_label, new_label in old_to_new_map.items():
            print(f"\n📦 Migrating emails from '{old_label}' → '{new_label}'")
            
            # 1️⃣ Get old label ID
            labels = self.get_labels()
            old_label_obj = next((l for l in labels if l['name'].lower() == old_label.lower()), None)
            if not old_label_obj:
                print(f"⚠️ Old label '{old_label}' not found, skipping.")
                continue
            old_label_id = old_label_obj['id']

            # 2️⃣ Ensure new label exists
            if not self.check_label_exists(new_label):
                self.create_label(new_label)
            new_label_id = next((l['id'] for l in self.get_labels() if l['name'].lower() == new_label.lower()), None)

            # 3️⃣ Fetch emails with old label
            try:
                results = self.service.users().messages().list(
                    userId='me',
                    labelIds=[old_label_id]
                ).execute()
                messages = results.get('messages', [])
                if not messages:
                    print(f"✅ No emails found under '{old_label}'")
                    continue

                print(f"📬 Found {len(messages)} emails under '{old_label}'")

                # 4️⃣ Move emails to new label
                for msg in messages:
                    self.service.users().messages().modify(
                        userId='me',
                        id=msg['id'],
                        body={'addLabelIds': [new_label_id], 'removeLabelIds': [old_label_id]}
                    ).execute()
                print(f"✅ Migrated {len(messages)} emails to '{new_label}'")

            except Exception as e:
                print(f"❌ Error migrating emails from '{old_label}': {e}")

            # 5️⃣ Delete old label
            try:
                self.service.users().labels().delete(userId='me', id=old_label_id).execute()
                print(f"🗑️ Deleted old label '{old_label}'")
            except Exception as e:
                print(f"❌ Error deleting old label '{old_label}': {e}")



In [12]:
# handler = GmailHandler()

In [13]:
# handler.get_labels()

In [14]:
# handler.get_unread_emails()

In [15]:
# handler.get_emails_since(hours=12, max_results=20)

In [29]:
# ================================
# Improved Email Classification Prompt
# ================================

EMAIL_CLASSIFICATION_PROMPT = """
You are a precise email classifier for job-related emails.
Your task is to assign the email to EXACTLY ONE of the following category keys:

{categories}

=========================
CATEGORY DEFINITIONS
=========================

application_confirmed:
    Confirmation that an application was received.

interview_request:
    Recruiter requesting to schedule an interview or providing interview scheduling links.

interview_reminder:
    Reminders or confirmations for upcoming interviews.

offer:
    Job offer emails, offer letters, verbal offers, or negotiation instructions.

rejected:
    Rejection emails stating the applicant is not moving forward.

assessment:
    Coding tests, online assessments, take-home assignments.

follow_up:
    Recruiter checking in, following up, asking for updates, or next-steps clarification.

job_alert:
    Job recommendations, alerts about openings, job board notifications.

newsletter:
    Company newsletters, weekly digests, marketing content.

spam:
    Irrelevant, suspicious, non-job content.

uncategorized:
    Use ONLY if the email clearly does not fit any category above.

=========================
CATEGORY PRIORITY
=========================
interview_request > interview_reminder > offer > rejected > assessment >
follow_up > application_confirmed > job_alert > newsletter > spam > uncategorized

=========================
EMAIL TO CLASSIFY
=========================

Subject: {subject}
Snippet: {snippet}

=========================
RESPONSE RULES
=========================
- Respond with ONLY the category key.
- No explanation.
- No extra text.
- No formatting.
Category:
"""


In [30]:
JOB_CATEGORIES = {
    "application_confirmed": {"label": "Applied ✓"},
    "interview_request": {"label": "Interview 📅"},
    "interview_reminder": {"label": "Interview Reminder ⏰"},
    "offer": {"label": "Job Offer 🎉"},
    "rejected": {"label": "Rejected ❌"},
    "assessment": {"label": "Assessment 📝"},
    "follow_up": {"label": "Follow-up 💬"},
    "job_alert": {"label": "Job Alert 🔔"},
    "newsletter": {"label": "Newsletter 📰"},
    "spam": {"label": "Spam 🗑️"},
    "uncategorized": {"label": "Other 📧"}
}


In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI

class EmailCategorizerUsingGemini:
    """Email categorizer using Google Gemini LLM"""
    
    def __init__(self,google_api_key, model_name="gemini-2.5-flash"):
        self.llm = ChatGoogleGenerativeAI(google_api_key=google_api_key,
            model=model_name)
        self.prompt_template = PromptTemplate(
            input_variables=["categories", "subject", "snippet"],
            template=EMAIL_CLASSIFICATION_PROMPT
        )
        self.chain = LLMChain(llm=self.llm, prompt=self.prompt_template)


    def categorize(self, emails):
        """Categorize a list of emails"""
        results = []
        categories_str = ", ".join(JOB_CATEGORIES.keys())

        for email in emails:
            subject = email.get("subject", "")
            snippet = email.get("snippet", "")
            try:
                response = self.chain.run(
                    subject=subject,
                    snippet=snippet,
                    categories=categories_str
                ).strip().lower()

                # match valid category or fallback
                category_key = (
                    response if response in JOB_CATEGORIES else "uncategorized"
                )
                label = JOB_CATEGORIES[category_key]["label"]

                email["category"] = category_key
                email["category_label"] = label
                results.append(email)

                print(f"📧 {subject[:60]} → {label}")

            except Exception as e:
                print(f"❌ Error classifying email '{subject}': {e}")

        return results


In [18]:
# from langchain_core.prompts import PromptTemplate
# from langchain_classic.chains import LLMChain
# from langchain_groq import ChatGroq
# from langchain_google_genai import ChatGoogleGenerativeAI




# class EmailCategorizer:
#     """Categorizes emails using Groq + Llama via LangChain.
#     Goes through a list of emails and classifies each into predefined job-related categories.
#     """

#     def __init__(self, groq_api_key, model_name="llama-3.1-8b-instant"):
#         self.model = ChatGroq(api_key=groq_api_key, model=model_name)
#         self.prompt = PromptTemplate(
#             input_variables=["subject", "snippet", "categories"],
#             template=EMAIL_CLASSIFICATION_PROMPT
#         )
#         self.chain = LLMChain(llm=self.model, prompt=self.prompt)

#     def categorize(self, emails):
#         """Categorize a list of emails"""
#         results = []
#         categories_str = ", ".join(JOB_CATEGORIES.keys())

#         for email in emails:
#             subject = email.get("subject", "")
#             snippet = email.get("snippet", "")
#             try:
#                 response = self.chain.run(
#                     subject=subject,
#                     snippet=snippet,
#                     categories=categories_str
#                 ).strip().lower()

#                 # match valid category or fallback
#                 category_key = (
#                     response if response in JOB_CATEGORIES else "uncategorized"
#                 )
#                 label = JOB_CATEGORIES[category_key]["label"]

#                 email["category"] = category_key
#                 email["category_label"] = label
#                 results.append(email)

#                 print(f"📧 {subject[:60]} → {label}")

#             except Exception as e:
#                 print(f"❌ Error classifying email '{subject}': {e}")

#         return results


In [8]:
class GmailLLMAgent:
    """
    Fetch emails from Gmail Classifier, classify them using LLM Class, 
    and apply labels based on the predicted category and update it in Gmail.

    """

    def __init__(self, gmail_handler: GmailHandler, categorizer: EmailCategorizer):
        self.gmail = gmail_handler
        self.categorizer = categorizer

    def process_emails(self, hours: int = 24, max_results: int = 5):
        """
        Fetch emails from the last 'hours' hours, classify, and label them.
        """
        emails = self.gmail.get_emails_since(hours=hours, max_results=max_results)
        if not emails:
            print("📭 No emails to process.")
            return

        print(f"📬 Processing {len(emails)} emails...\n")

        # 1️⃣ Classify emails
        categorized_emails = self.categorizer.categorize(emails)

        # 2️⃣ Apply labels
        for email in categorized_emails:
            label_name = email.get("category_label", "Other 📧")

            # create label if not exists
            if not self.gmail.check_label_exists(label_name):
                self.gmail.create_label(label_name)

            # apply label
            try:
                self.gmail.service.users().messages().modify(
                    userId='me',
                    id=email['id'],
                    body={'addLabelIds': [self._get_label_id(label_name)],
                          'removeLabelIds': []}  # optional: remove UNREAD
                ).execute()
                print(f"✅ Labeled: {email['subject'][:50]} → {label_name}")
            except Exception as e:
                print(f"❌ Error applying label to '{email['subject']}': {e}")

    def _get_label_id(self, label_name: str):
        """Return Gmail label ID, create if it doesn't exist"""
        labels = self.gmail.get_labels()
        for label in labels:
            if label['name'].lower() == label_name.lower():
                return label['id']
        # create if not found
        label = self.gmail.create_label(label_name)
        return label['id'] if label else None


In [9]:
import os
from dotenv import load_dotenv
load_dotenv()


# Initialize Gmail handler
gmail_handler = GmailHandler()

# Initialize Groq email categorizer
groq_api_key = os.getenv("GROQ_API_KEY")
categorizer = EmailCategorizer(groq_api_key)

# Create LLM Agent
agent = GmailLLMAgent(gmail_handler, categorizer)

# Process emails from last 24 hours
agent.process_emails(hours=24, max_results=50)


creds {"token": "ya29.a0ATi6K2u8ae8fTAyDuVZO3_jMmcMd2Ifm_RHySU-yxgzUWPpi-F62Ge9FsKR26yD1gq8p7dLuglDIOwOjTE1HUNQJ4VOW9aa9NjzbQ0YsSRcUqqczDhYWoRz7zHM532QQm4_Jxd0jZimg6kds_Y7ToqE244raJH8p77r7vs9AlCuxaWUhSj7t1mX9c_yoOIApWdg_GqYXaCgYKAS0SARMSFQHGX2MiFZukLWjbupDNNX7H85miQw0207", "refresh_token": "1//05TrQi9MqXxoTCgYIARAAGAUSNwF-L9IrtueZ2vud33vXcr9aQhXc0R1T7ekq0Lv2GFgN_cvw8k0yDL-GLPc46lwwoOOwCdir4lc", "token_uri": "https://oauth2.googleapis.com/token", "client_id": "643813721017-2err0bl1c4ueihcvsgbqm7cckvulsip6.apps.googleusercontent.com", "client_secret": "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", "scopes": ["https://www.googleapis.com/auth/gmail.modify"], "universe_domain": "googleapis.com", "account": "", "expiry": "2025-11-14T18:43:46Z"}


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_23044\3484252649.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


📬 Found 50 emails from the last 24 hours:

📧 From: LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>
   Subject: “Canada”: Toronto Paramedic Services (Formerly Toronto EMS) - OPERATIONS SUPPORT OFFICER (Ceremonial Documents), STRATEGIC PROTOCOL AND EXTERNAL RELATIONS
   Labels: UNREAD, CATEGORY_UPDATES, INBOX
   Snippet: Toronto Paramedic Services (Formerly Toronto EMS) OPERATIONS SUPPORT OFFICER (Ce...

📧 From: Microsoft Reactor Toronto <info@email.meetup.com>
   Subject: 📅 Just scheduled: Maintaining reliable codebases with GitHub Code Quality
   Labels: UNREAD, IMPORTANT, CATEGORY_UPDATES, INBOX
   Snippet: Microsoft Reactor Toronto invites you to a new event ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌...

📧 From: "BestBuy.ca" <newsletter@e.bestbuy.ca>
   Subject: Call of Duty: Black Ops 7 available now.
   Labels: CATEGORY_PROMOTIONS, UNREAD, INBOX
   Snippet: Plus, save up to 25% on select gaming accessories. ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿ ͏ ‌ ﻿...

📧 From: AI Tinkerers - Toronto <toronto-events@mail

C:\Users\Abhishek\AppData\Local\Temp\ipykernel_23044\3484252649.py:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = self.chain.run(


📧 “Canada”: Toronto Paramedic Services (Formerly Toronto EMS)  → Other 📧
📧 📅 Just scheduled: Maintaining reliable codebases with GitHub → Job Alert 🔔
📧 Call of Duty: Black Ops 7 available now. → Job Alert 🔔
📧 AI Tinkerers Toronto x Two Small Fish - VIP Dinner! → Job Alert 🔔
📧 Is your potential boss toxic? → Newsletter 📰
📧 Andrew Bryan posted in Canadian Networking. → Job Alert 🔔
📧 Andrew Bryan posted in Canadian Networking. → Job Alert 🔔
📧 ✈ Book Budget-Friendly Deals! → Newsletter 📰
📧 Thank you for applying to DoorDash → Job Alert 🔔
📧 [Correction] Updated Link to All Course Livestreams → Other 📧
📧 New Accounting, Audit, SWE Roles for '26 → Job Alert 🔔
📧 30+ new jobs in Canada → Job Alert 🔔
📧 ACT NOW! UP TO 35% OFF — BLACK FRIDAY DEALS 🎉 → Spam 🗑️
📧 🎟️ TORONTO! Tickets for RAW are On Sale Now! → Newsletter 📰
📧 💼 3 new job opportunities posted in the past 24 hours → Job Alert 🔔
📧 Cursor $29B valuation 💰, Google Code Wiki 👨‍💻, advanced Nano → Newsletter 📰
📧 Shop, Savour, and Celebrate – 

In [12]:
handler = GmailHandler()
emails = handler.get_emails_since(hours=48, max_results=10)

creds {"token": "ya29.a0ATi6K2uWR3QoHn6e9C3Fzkz4CxaR-S-jo80RBU1vXelIiMXfKftMzeGvtGP68SzoFLHlijnaCHH1kXn3-4r8kZ93b4HmHPnwfWZs56RkW_iSQPpLleVvjCJNeJWLdmUzxqICeNVm8CXdxgi5dyqYEthjPGHCMG8kCSuiPVBzQQOZaH_SXGnzeQ6alCcIkWzUnC3bqtEnaCgYKAekSARMSFQHGX2Mib2etkJXuatNh2v0hKQS37Q0207", "refresh_token": "1//05TrQi9MqXxoTCgYIARAAGAUSNwF-L9IrtueZ2vud33vXcr9aQhXc0R1T7ekq0Lv2GFgN_cvw8k0yDL-GLPc46lwwoOOwCdir4lc", "token_uri": "https://oauth2.googleapis.com/token", "client_id": "643813721017-2err0bl1c4ueihcvsgbqm7cckvulsip6.apps.googleusercontent.com", "client_secret": "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", "scopes": ["https://www.googleapis.com/auth/gmail.modify"], "universe_domain": "googleapis.com", "account": "", "expiry": "2025-11-16T19:11:58Z"}
📬 Found 10 emails from the last 48 hours:

📧 From: LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>
   Subject: Microsoft Power Platform Community is hiring in Canada
   Labels: UNREAD, CATEGORY_UPDATES, INBOX
   Snippet: View jobs in Canada ͏ ͏ ͏ ͏ ͏ ͏ ͏

In [26]:
import os
from dotenv import load_dotenv
load_dotenv()

categorizer = EmailCategorizer(google_api_key=os.getenv("GOOGLE_API_KEY"))


In [27]:
categorized_emails = categorizer.categorize(emails)

C:\Users\Abhishek\AppData\Local\Temp\ipykernel_22540\4052481395.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = self.chain.run(


📧 Microsoft Power Platform Community is hiring in Canada → Job Alert 🔔
📧 i am 13 yrs old in 8 th grade..my mom usually talks to this  → Spam 🗑️
📧 From Self-Doubt to a $195K Offer at NVIDIA – Amya’s Comeback → Newsletter 📰
📧 New jobs: Senior Software Engineer at Othelia Technologies a → Job Alert 🔔
📧 [Slack] Single sign-on enabled → Other 📧
📧 Companies are now considering human managers for AI agents → Newsletter 📰
📧 📍 AI Tinkerers TONIGHT 6 PM - Directions + Parking → Newsletter 📰
📧 Maria Dixon posted in Toronto Singles in the city (4000+) → Spam 🗑️
📧 Maria Dixon posted in Toronto AI, ML and Data Meetup Group → Spam 🗑️
📧 Google’s New AI Trio Changes Everything → Newsletter 📰
